In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('datos/credit_card.csv')
y = pd.read_csv('datos/credit_card_label.csv', usecols=["label"])

x = df.drop(columns=["Ind_ID"])
x = pd.get_dummies(x)
display(x.head())


rf = HistGradientBoostingClassifier()
param_grid = {
    'max_depth': [None,1,2,3,4,5,6,7,8,9,10,11],
    
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=StratifiedKFold(n_splits=5, shuffle=True),
                           scoring='f1', n_jobs=-1)




grid_search.fit(x, y.values.ravel())
y_pred = grid_search.predict(x)
print("Mejores hiperparámetros encontrados:", grid_search.best_params_)
print("Mejor accuracy promedio (CV):", grid_search.best_score_)

,CHILDREN,Annual_income,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Family_Members,GENDER_F,...,Type_Occupation_Laborers,Type_Occupation_Low-skill Laborers,Type_Occupation_Managers,Type_Occupation_Medicine staff,Type_Occupation_Private service staff,Type_Occupation_Realty agents,Type_Occupation_Sales staff,Type_Occupation_Secretaries,Type_Occupation_Security staff,Type_Occupation_Waiters/barmen staff
0,0,180000.0,-18772.0,365243,1,0,0,0,2,False,...,False,False,False,False,False,False,False,False,False,False
1,0,315000.0,-13557.0,-586,1,1,1,0,2,True,...,False,False,False,False,False,False,False,False,False,False
2,0,315000.0,NaN,-586,1,1,1,0,2,True,...,False,False,False,False,False,False,False,False,False,False
3,0,NaN,-13557.0,-586,1,1,1,0,2,True,...,False,False,False,False,False,False,False,False,False,False
4,0,315000.0,-13557.0,-586,1,1,1,0,2,True,...,False,False,False,False,False,False,False,False,False,False


Mejores hiperparámetros encontrados: {'max_depth': None}
Mejor accuracy promedio (CV): 0.5680409115872734
